In [1]:
!python --version

Python 3.11.6


In [2]:
# set openAI API key
import os
from openai import OpenAI
import pprint
import json
import math
from IPython.display import Audio, display
from pydub import AudioSegment
import io
import time
import sys
from pathlib import Path
__file__ = os.path.dirname(os.path.abspath('__file__'))
client = OpenAI()

In [3]:
response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input="The quick brown fox jumped over the lazy dog."
)
speech_file_path = Path(__file__) / "speech.mp3"
response.stream_to_file(speech_file_path)

#### srt and vtt is like cc transcripts in videos with timestamp on each line

In [4]:
audio_file = open("speech.mp3", "rb")

# b_handle = io.BytesIO()
# b_handle.write(response.content)
# b_handle.seek(0)
# audio_file = io.BufferedReader(b_handle)

transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  response_format="srt"
)

In [5]:
transcript

'1\n00:00:00,000 --> 00:00:02,800\nThe quick brown fox jumped over the lazy dog.\n\n\n'

#### verbose_json has all the details in it

In [6]:
audio_file = open("speech.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  response_format="verbose_json"
)

In [7]:
transcript.model_dump()

{'text': 'The quick brown fox jumped over the lazy dog.',
 'task': 'transcribe',
 'language': 'english',
 'duration': 2.85,
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 2.8000000000000003,
   'text': ' The quick brown fox jumped over the lazy dog.',
   'tokens': [50364,
    440,
    1702,
    6292,
    21026,
    13864,
    670,
    264,
    14847,
    3000,
    13,
    50504],
   'temperature': 0.0,
   'avg_logprob': -0.4547843933105469,
   'compression_ratio': 0.8653846153846154,
   'no_speech_prob': 0.0002566356270108372}]}

#### text and json are just simple responses

In [8]:
audio_file = open("speech.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  response_format="text"
)

In [9]:
transcript

'The quick brown fox jumped over the lazy dog.\n'

### Talk directly in notebook

In [16]:
from ipywebrtc import AudioRecorder, CameraStream
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [11]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav file.wav -y -hide_banner -loglevel panic

In [12]:
audio_file = open("file.wav", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  response_format="text"
)

In [14]:
transcript

'Hi, my name is Jack. How can I help you today?\n'